Credit: Ms Ong

# Task 4.1

In [ ]:
import pymongo
import csv

# connect to mongodb
client = pymongo.MongoClient("127.0.0.1", 27017)
clinic = client.get_database("Clinic")
patient = clinic.get_collection("Patient")

# open and read from file
with open ("PATIENTS.txt", "r") as patients_file:
    patients_data = csv.reader(patients_file)
    patients_data = [data for data in patients_data]
   
    for i in range(len(patients_data)):
        data = patients_data[i]
        patient_dict = {}
        patient_dict["PatientID"] = int(data[0]) # integer
        patient_dict["Name"] = data[1]
        patient_dict["AppointmentDate"] = data[2]
        patient_dict["AppointmentStartTime"] = data[3]
        patient_dict["DoctorAssigned"] = data[4]
        patient_dict["RoomNumber"] = data[5]
        patient_dict["AmountCharged"] = float(data[6])
        patient.insert_one(patient_dict)

print("Finished insertion!")
client.close()

Finished insertion!


# Task 4.2

In [ ]:
import pymongo
import random

def id_generator ():
    client = pymongo.MongoClient("127.0.0.1", 27017)
    clinic = client.get_database("Clinic")
    patient = clinic.get_collection("Patient")

    ids = patient.find({}, {"PatientID": 1})
    unique_ids = set()
    for id_ in ids:
    #     print(id_["PatientID"])
        unique_ids.add(id_["PatientID"])
    gen_id = False
    while not gen_id:
        new_id = random.randint(1, 100)
        curr_length = len(unique_ids)
        unique_ids.add(new_id)
        if len(unique_ids) > curr_length:
            gen_id = True
    client.close()
    return new_id

print("Enter new patient:")
valids = {"name": False, "appt_date": False, "appt_start_time": False, "doctor_assigned": False}
while not valids["name"]:
    name = input("Enter patient's name: ")
    valid_name = True
    for letter in name:
        if (ord(letter) != ord(" ")) and (not ord("A") <= ord(letter) <= ord("Z")) and (not ord("a") <= ord(letter) <= ord("z")):
            valid_name = False
            print("Expected only space and letters")
            break
    valids["name"] = valid_name
while not valids["appt_date"]:
    valid_appt_date = True
    appt_date = input("Enter appointment date in DDMMYYYY format: ")
    if len(appt_date) != 8: # length check
        print("Expected DDMMYYYY format!")
        valid_appt_date = False
    else:
        temp_appt_date = appt_date
        try:
            temp_appt_date = int(temp_appt_date) # data type check
        except:
            print("Expected integer format in DDMMYYYY!")
            valid_appt_date = False
    if valid_appt_date:
        # check range
        days = int(appt_date[:2])
        months = int(appt_date[2:4])
        years = int(appt_date[4:])
        # even months only have 30 days
        if (not 1 <= months <= 12 or not 0 <= days <= 31 or not years >= 2020
            or (months % 2 == 0 and days > 30) or (months == 2 and days > 29)): # special case for feb
            print("Enter valid date!")
            valid_appt_date = False
    valids["appt_date"] = valid_appt_date
while not valids["appt_start_time"]:
    valid_appt_start_time = True
    appt_start_time = input("Enter appointment start time in 24hr HHMM format: ")
    if len(appt_start_time) != 4: # length check
        print("Expected a HHMM format!")
        valid_appt_start_time = False
    else:
        temp_appt_start_time = appt_start_time
        try:
            temp_appt_start_time = int(appt_start_time)
        except:
            print("Expected integer format in HHMM!")
            valid_appt_start_time = False
    if valid_appt_start_time:
        hrs = int(appt_start_time[:2])
        mins = int(appt_start_time[2:])
        if not 0 <= hrs <= 24 or not 0 <= mins <= 60:
            print("Expected a valid time!")
            valid_appt_start_time = False
    valids["appt_start_time"] = valid_appt_start_time

while not valids["doctor_assigned"]:
    doctor_assigned = input("Enter doctor assigned: ")
    if doctor_assigned == "Doctor 1" or doctor_assigned == "Doctor 2" or doctor_assigned == "Doctor 3":
        # data check
        valids["doctor_assigned"] = True
    else:
        print("Expected either Doctor 1, Doctor 2 or Doctor 3!")
    
            
print("-" * 30)

# connect to mongodb
client = pymongo.MongoClient("127.0.0.1", 27017)
clinic = client.get_database("Clinic")
patient = clinic.get_collection("Patient")
# check if is existing patient
patient_data = patient.find_one({"Name": name}, {"PatientID": 1})
if patient_data is None:
    # generate new id
    print("Generating new id...", end="")
    patient_id = id_generator()
else:
    patient_id = patient_data["PatientID"]
    
print("Keying in data...")
patient.insert_one({"PatientID": patient_id, "Name": name, "AppointmentDate": appt_date,
                   "AppointmentStartTime": appt_start_time, "DoctorAssigned": doctor_assigned})
print("Finished insertion!")
print("-" * 50)
client.close()

Enter new patient:
Enter patient's name: emily oooo 99
Expected only space and letters
Enter patient's name: emily ong
Enter appointment date in DDMMYYYY format: 23rync32,o;cew
Expected DDMMYYYY format!
Enter appointment date in DDMMYYYY format: 
Expected DDMMYYYY format!
Enter appointment date in DDMMYYYY format: 09081999
Enter valid date!
Enter appointment date in DDMMYYYY format: 09092020
Enter appointment start time in 24hr HHMM format: 9988
Expected a valid time!
Enter appointment start time in 24hr HHMM format: 2666
Expected a valid time!
Enter appointment start time in 24hr HHMM format: jjjj
Expected integer format in HHMM!
Enter appointment start time in 24hr HHMM format: 
Expected a HHMM format!
Enter appointment start time in 24hr HHMM format: 200
Expected a HHMM format!
Enter appointment start time in 24hr HHMM format: 2000
Enter doctor assigned: Doctor 44
Expected either Doctor 1, Doctor 2 or Doctor 3!
Enter doctor assigned: Doctor 1
------------------------------
Generatin

# Task 4.3

In [ ]:
import pymongo

# connect to mongodb
client = pymongo.MongoClient("127.0.0.1", 27017)
clinic = client.get_database("Clinic")
patient = clinic.get_collection("Patient")

# handle double bookings
patient_data = patient.find({}).sort("Name")
patient_bookings = {} # key:name, value:bookings
for data in patient_data:
    name = data["Name"]
    if name not in patient_bookings:
        patient_bookings[name] = set()
    curr_booking = len(patient_bookings[name])
    patient_bookings[name].add((data["AppointmentDate"], data["AppointmentStartTime"]))
    # check for duplicates
    if len(patient_bookings) == curr_booking:
        print("Deleting duplicates...")
        patient.delete_one(data)
        

# handle appt clashes
# clash occurs when we have same doctor-appt_start_time-appt_date
patient_data = patient.find().sort("Name")
available_timings = {}
for i in range (10, 21): # assume clinic works from 10am to 8pm and each appt is 1hr
    available_timings[str(i)+"00"] = 0
patient_appts = {} # key:name, value:bookings
for data in patient_data:
    name = data["Name"]
    if name not in patient_appts:
        patient_appts[name] = set()
    curr_appts = len(patient_appts[name])
    patient_appts[name].add((data["DoctorAssigned"], data["AppointmentDate"], data["AppointmentStartTime"]))
    if len(patient_appts[name]) == curr_appts:
        print("Rescheduling...")
        # get the doctor's schedule
        doctor_schedule = patient.find({"DoctorAssigned": data["DoctorAssigned"], 
                                        "AppointmentDate": data["AppointmentDate"]}, {"AppointmentStartTime": 1}).sort("AppointmentStartTime")
        for schedule in doctor_schedule:
            available_timings[schedule["AppointmentStartTime"]] = 1
        for timing in available_timings:
            if not available_timings[timing]:
                print(f"Reassigning to {timing}...")
                patient.update_one(data, {"$set": {"AppointmentStartTime": timing}})
        
client.close()

Deleting duplicates...
Deleting duplicates...
Deleting duplicates...


In [ ]:
print(ord(" "))
print(ord("a"))
print(ord("z"))
print(ord("A"))
print(ord("Z"))

32
97
122
65
90


# Task 4.4

In [ ]:
import pymongo

# connect to mongodb
client = pymongo.MongoClient("127.0.0.1", 27017)
clinic = client.get_database("Clinic")
patient = clinic.get_collection("Patient")

# determine top-3 paying patients
# patients = patient.find({}, {"Name": 1, "AmountCharged": 1}).sort("AmountCharged", pymongo.DESCENDING)
patients = patient.aggregate([
    {"$group":
     {"_id": "$Name",
     "TotalCharged": {"$sum": "$AmountCharged"}
     }
    }
])
count = 0
print("The top-3 paying patients are...")

patient_data = []
for data in patients:
    patient_data.append([data["TotalCharged"], data["_id"]])

# implement sorting algorithm
def partition (arr, start, end):
    pivot = arr[end][0]
    pi = start - 1
    for i in range (start, end):
        if arr[i][0] > pivot:
            pi += 1
            arr[pi], arr[i] = arr[i], arr[pi]
    pi += 1
    arr[pi], arr[end] = arr[end], arr[pi]
    return pi
    
def quick_sort (arr, start, end):
    if start < end:
        pi = partition(arr, start, end)
        quick_sort(arr, start, pi - 1)
        quick_sort(arr, pi + 1, end)
    return arr

patient_data = quick_sort(patient_data, 0, len(patient_data) - 1)
    
for data in patient_data:
    print(f"{count + 1}. {data[1]} (${data[0]})")
    count += 1
    if count == 3:
        break
    
client.close()

The top-3 paying patients are...
1. John Ng ($990.0)
2. Richard Lee ($670.0)
3. Lily Soh ($450.0)


In [ ]:
import pymongo

# connect to mongodb
client = pymongo.MongoClient("127.0.0.1", 27017)
clinic = client.get_database("Clinic")
patient = clinic.get_collection("Patient")

patients = patient.find({}).sort("PatientName")
patient_names = set()

duplicated_patients = set()

for data in patients:
    name = data["Name"]
    patient_id = data["PatientID"]
    curr_length = len(patient_names)
    patient_names.add(name)
    if curr_length == len(patient_names):
        duplicated_patients.add((name, patient_id))

print("The patients are...")
print("Name\tPatient ID")
for duplicated_patient in duplicated_patients:
    print(f"{duplicated_patient[0]}\t{duplicated_patient[1]}")
client.close()

The patients are...
Name	Patient ID
John Ng	2
Lily Soh	3
Kyle Teo	5
John Ng	4
Richard Lee	6
Peter Tan	1
